# Boundary Value Problems
## Christina Lee
## Numerics
## Prerequisites: 1D Root Finding, ODE Solvers

In the post on solving Ordinary Differential Equations, we specified the all the initial conditions at the beginning of the problem.

But not all problems will be posed that way.  

Suppose you're firing a projectile.  You know where the cannon is and where you want the explosive to eventually land, and you need to determine what angle and velocity to aim.  That's the problem we'll be applying the method to here.  Though it might seem simple now, this problem drove much of the defense department funding in computers in the early years.  

In [1]:
using Plots
plotlyjs()

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/64a7ce9663f24dbec2e2f9486130583ce8a809fa-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/64a7ce9663f24dbec2e2f9486130583ce8a809fa-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

Plots.PlotlyJSBackend()

We'll just copy over our RK4 and Solver routines from the ODE post.  You could also just a package like DifferentialEquations.jl, but since we are working with the numerical pecularities in this post instead of wrapping the numerics under the hood to solve a physics problem, I prefer to use my own routines.

In [2]:
function RK4(f::Array{Function,1},t0::Float64,x::Array{Float64,1},h::Float64)
    d=length(f)
    
    hk1=zeros(Float64,length(x))
    hk2=zeros(Float64,length(x))
    hk3=zeros(Float64,length(x))
    hk4=zeros(Float64,length(x))
    
    for ii in 1:d
        hk1[ii]=h*f[ii](t0,x)
    end
    for ii in 1:d
        hk2[ii]=h*f[ii](t0+h/2,x+hk1/2)
    end
    for ii in 1:d
        hk3[ii]=h*f[ii](t0+h/2,x+hk2/2) 
    end
    for ii in 1:d
        hk4[ii]=h*f[ii](t0+h,x+hk3)
    end
    
    return t0+h,x+(hk1+2*hk2+2*hk3+hk4)/6
end

function Solver(f::Array{Function,1},Method::Function,t0::Float64,
        x0::Array{Float64,1},h::Float64,N::Int64)
    d=length(f)
    ts=zeros(Float64,N+1)
    xs=zeros(Float64,d,N+1)
    
    ts[1]=t0
    xs[:,1]=x0
    
    for i in 2:(N+1)
        ts[i],xs[:,i]=Method(f,ts[i-1],xs[:,i-1],h)
    end
    
    return ts,xs
end

Solver (generic function with 1 method)

Today, we are focusing at the point at where the projectile hits the ground again.  While we could pull this data out after performing a full computation, for computational efficiency and ease of use, I wrote up a modified version of the solver that breaks and returns the current evaluation when the path drops below  the y-axis.  This way we aren't evaluating useless things and are important data points will be conviently located at the end of the arrray.

In [3]:
function Solver_mod(f::Array{Function,1},Method::Function,t0::Float64,
        x0::Array{Float64,1},h::Float64,N::Int64)
    d=length(f)
    ts=zeros(Float64,N+1)
    xs=zeros(Float64,d,N+1)
    
    ts[1]=t0
    xs[:,1]=x0 
    
    for i in 2:(N+1)
        ts[i],xs[:,i]=Method(f,ts[i-1],xs[:,i-1],h)
        
        if xs[2,i]<0
            println("Hit ground at t = ",ts[i-1]," and x = ",xs[1,i-1])
            return ts[1:(i-1)],xs[:,1:(i-1)]
        end
    end
    
    return ts,xs
end

Solver_mod (generic function with 1 method)

$$
\frac{\text{d} x_i}{\text{d} t}  = v_i
$$
$$
\frac{\text{d} v_i }{\text{d}t} = g_i - A |v|^2
$$

In [4]:
A=0.01
g=-1.

-1.0

In [5]:
function dx(t::Float64,x::Array{Float64,1})
    return x[3]
end
function dy(t::Float64,x::Array{Float64,1})
    return x[4]
end
function dvx(t::Float64,x::Array{Float64,1})
    return -A*(x[3]^2+x[4]^2)
end
function dvy(t::Float64,x::Array{Float64,1})
    return g-A*(x[3]^2+x[4]^2)
end

du=Function[]
push!(du,dx)
push!(du,dy)
push!(du,dvx)
push!(du,dvy)

4-element Array{Function,1}:
 dx 
 dy 
 dvx
 dvy

In [27]:
v=.8
θ=π/4


function u(vθ::Array{Float64})
    return [0.,0.,vθ[1]*cos(vθ[2]),vθ[1]*sin(vθ[2])]
end

u (generic function with 1 method)

In [31]:
function Measure_u0(u0)
    ts,xs=Solver_mod(du,RK4,0.0,u(u0),0.01,1000)
    diff=xs[1,end]-goal
    println("Diff \t",diff)
    return diff
end

Measure_u0 (generic function with 1 method)

In [14]:
function Secant_Method(x::Array)
    xnew= x[2] - x[4]*(x[2]-x[1])/(x[4]-x[3])
    return [x[2],xnew,x[4],Measure_u0(xnew)]
end

Secant_Method (generic function with 2 methods)

In [8]:
goal=.75

u01=[1.,π/4]
ts_u01,xs_u01=Solver_mod(du,RK4,0.0,u(u01),0.01,1000)
f_u01=xs_u01[1,end]-goal

u02=[.95,π/4.2]
ts_u02,xs_u02=Solver_mod(du,RK4,0.0,u(u02),0.01,1000)
f_u02=xs_u02[1,end]-goal

Hit ground at t = 1.400000000000001 and x = 0.9834632270332824
Hit ground at t = 1.280000000000001 and x = 0.8863108256757052


0.13631082567570518

In [32]:
ui=[u01,u02,f_u01,f_u02]                          

4-element Array{Any,1}:
  [1.0, 0.785398]   
  [0.95, 0.747998]  
 0.23346322703328237
 0.13631082567570518

In [62]:
ui=Secant_Method(ui)

Hit ground at t = 1.1100000000000008 and x = 0.7458549557737626
Diff 	-0.004145044226237404


4-element Array{Any,1}:
   [0.871896, 0.689577]
   [0.878591, 0.694585]
 -0.013391182921252187 
 -0.004145044226237404 

In [ ]:
function Iterate(inputs)
    up=Secand_Method()
    return up,f_up

In [9]:
ui=Secant_Method(u01,u02,f_u01,f_u02)
ts_ui,xs_ui=Solver_mod(du,RK4,0.0,u(ui),0.01,1000)
f_ui=xs_ui[1,end]-goal

Hit ground at t = 1.1200000000000008 and x = 0.7530240094064591


0.0030240094064590828

In [10]:
uii=Secant_Method(u02,ui,f_u02,f_ui)
ts_uii,xs_uii=Solver_mod(du,RK4,0.0,u(uii),0.01,1000)
f_uii=xs_uii[1,end]-goal

Hit ground at t = 1.1100000000000008 and x = 0.7457269869150551


-0.004273013084944877

In [11]:
uiii=Secant_Method(ui,uii,f_ui,f_uii)
ts_uiii,xs_uiii=Solver_mod(du,RK4,0.0,u(uiii),0.01,1000)
f_uiii=xs_uiii[1,end]-goal

Hit ground at t = 1.1100000000000008 and x = 0.7460813460051908


-0.003918653994809174

In [16]:
plot(xs_u01[1,:],xs_u01[2,:],label="Guess 1")
plot!(xs_u02[1,:],xs_u02[2,:],label="Guess 2")
plot!(xs_ui[1,:],xs_ui[2,:],label="Guessed Root")

scatter!([goal],[0.0],markersize=10,label="Target")
plot!(xlabel="x",ylabel="y",title="Fitting to a Boundary Condition")

WebIO.mount(this.previousSibling,{"props":{},"nodeType":"Scope","type":"node","instanceArgs":{"imports":{"data":[{"name":"Plotly","type":"js","url":"/assetserver/4930ed37c0789ac96a7f541889c15c8e885bc346-plotly-latest.min.js"},{"name":null,"type":"js","url":"/assetserver/caa940a2b0af3a19ea9eeab60526f9e75e786d4e-plotly_webio.bundle.js"}],"type":"async_block"},"id":"scope-515af4cb-1641-4fd5-b805-c3e182ca0a28","handlers":{"_toImage":[(function (options){return this.Plotly.toImage(this.plotElem,options).then((function (data){return WebIO.setval({"name":"image","scope":"scope-515af4cb-1641-4fd5-b805-c3e182ca0a28","id":"ob_05","type":"observable"},data)}))})],"__get_gd_contents":[(function (prop){prop=="data" ? (WebIO.setval({"name":"__gd_contents","scope":"scope-515af4cb-1641-4fd5-b805-c3e182ca0a28","id":"ob_06","type":"observable"},this.plotElem.data)) : undefined; return prop=="layout" ? (WebIO.setval({"name":"__gd_contents","scope":"scope-515af4cb-1641-4fd5-b805-c3e182ca0a28","id":"ob_06","type":"observable"},this.plotElem.layout)) : undefined})],"_promises":{"importsLoaded":[(function (Plotly){var gd=this.dom.querySelector("#plot-9e203803-a377-49a0-9443-15d190369643"); this.plotElem=gd; this.Plotly=Plotly; (window.Blink!==undefined) ? (gd.style.width="100%", gd.style.height="100vh", gd.style.marginLeft="0%", gd.style.marginTop="0vh") : undefined; window.onresize=(function (){return Plotly.Plots.resize(gd)}); Plotly.newPlot(gd,[],{"margin":{"l":50,"b":50,"r":50,"t":60}},{}); gd.on("plotly_hover",(function (data){var filtered_data=WebIO.PlotlyCommands.filterEventData(gd,data,"hover"); return !(filtered_data.isnil) ? (WebIO.setval({"name":"hover","scope":"scope-515af4cb-1641-4fd5-b805-c3e182ca0a28","id":"ob_01","type":"observable"},filtered_data.out)) : undefined})); gd.on("plotly_unhover",(function (){return WebIO.setval({"name":"hover","scope":"scope-515af4cb-1641-4fd5-b805-c3e182ca0a28","id":"ob_01","type":"observable"},{})})); gd.on("plotly_selected",(function (data){var filtered_data=WebIO.PlotlyCommands.filterEventData(gd,data,"selected"); return !(filtered_data.isnil) ? (WebIO.setval({"name":"selected","scope":"scope-515af4cb-1641-4fd5-b805-c3e182ca0a28","id":"ob_02","type":"observable"},filtered_data.out)) : undefined})); gd.on("plotly_deselect",(function (){return WebIO.setval({"name":"selected","scope":"scope-515af4cb-1641-4fd5-b805-c3e182ca0a28","id":"ob_02","type":"observable"},{})})); gd.on("plotly_relayout",(function (data){var filtered_data=WebIO.PlotlyCommands.filterEventData(gd,data,"relayout"); return !(filtered_data.isnil) ? (WebIO.setval({"name":"relayout","scope":"scope-515af4cb-1641-4fd5-b805-c3e182ca0a28","id":"ob_04","type":"observable"},filtered_data.out)) : undefined})); return gd.on("plotly_click",(function (data){var filtered_data=WebIO.PlotlyCommands.filterEventData(gd,data,"click"); return !(filtered_data.isnil) ? (WebIO.setval({"name":"click","scope":"scope-515af4cb-1641-4fd5-b805-c3e182ca0a28","id":"ob_03","type":"observable"},filtered_data.out)) : undefined}))})]},"_downloadImage":[(function (options){return this.Plotly.downloadImage(this.plotElem,options)})],"_commands":[(function (args){var fn=args.shift(); var elem=this.plotElem; var Plotly=this.Plotly; args.unshift(elem); return Plotly[fn].apply(this,args)})]},"systemjs_options":null,"observables":{"_toImage":{"sync":false,"id":"ob_08","value":{}},"hover":{"sync":false,"id":"ob_01","value":{}},"selected":{"sync":false,"id":"ob_02","value":{}},"__gd_contents":{"sync":false,"id":"ob_06","value":{}},"click":{"sync":false,"id":"ob_03","value":{}},"image":{"sync":true,"id":"ob_05","value":""},"__get_gd_contents":{"sync":false,"id":"ob_10","value":""},"_downloadImage":{"sync":false,"id":"ob_09","value":{}},"relayout":{"sync":false,"id":"ob_04","value":{}},"_commands":{"sync":false,"id":"ob_07","value":["relayout",{"showlegend":true,"xaxis":{"showticklabels":true,"gridwidth":0.5,"tickvals":[0.0,0.25,0.5,0.75,1.0],"visible":true,"ticks":"inside","range"